In [1]:
import numpy as np
import pandas as pd
import datetime

# Загрузка старых

In [2]:
old_table = pd.read_csv('/kaggle/input/mmvb-tovar/data.csv')
old_table.set_index(['<TICKER>','<DATE>'],inplace=True)
old_table

<CLOSE>  <VOL>
<TICKER>         <DATE>                    
@Brent Crude Oil 2020-04-13   31.830  41250
                 2020-04-14   30.050  35208
                 2020-04-15   28.000  35563
                 2020-04-16   28.300  34524
                 2020-04-17   28.350  36320
...                              ...    ...
Silver           2023-01-03   23.963      0
                 2023-01-04   23.776      0
                 2023-01-05   23.228      0
                 2023-01-06   23.827      0
                 2023-01-07   23.827      0

[8997 rows x 2 columns]

In [3]:
# https://mfd.ru/export/?groupId=2&selectedTickers=1*,2*,3*,4*,170*,1878,1924,1925,1926#Tickers=1*%2C2*%2C3*%2C4*%2C170*%2C1878%2C1924%2C1925%2C1926&Alias=false&Period=7&timeframeValue=1&timeframeDatePart=day&StartDate=06.01.2023&EndDate=06.01.2023&SaveFormat=0&SaveMode=0&FileName=mfdexport_1day_06012023_06012023.txt&FieldSeparator=%2C&DecimalSeparator=.&TimeFormat=HHmmss&DateFormatCustom=yyyy-MM-dd&AddHeader=true&RecordFormat=2&Fill=false

def get_new_table(delta_start=-10,delta_end=0):
    end_date = (datetime.datetime.today() + datetime.timedelta(days=delta_end)).strftime('%d.%m.%Y')
    start_date = (datetime.datetime.today() + datetime.timedelta(days=delta_start)).strftime('%d.%m.%Y')
    url = "https://mfd.ru/export/handler.ashx/mfdexport_1day.txt?TickerGroup=2&Tickers=1*%2C2*%2C3*%2C4*%2C170*%2C1878%2C1924%2C1925%2C1926&Alias=false&Period=7&timeframeValue=1&timeframeDatePart=day&StartDate=" + start_date + "&EndDate=" + end_date + "&SaveFormat=0&SaveMode=0&FileName=mfdexport_1day.txt&FieldSeparator=%253b&DecimalSeparator=.&DateFormat=yyyy-MM-dd&TimeFormat=HHmmss&DateFormatCustom=&TimeFormatCustom=&AddHeader=true&RecordFormat=2&Fill=true"
    table = pd.read_csv(url,sep=";")
    new_table = table[["<TICKER>","<DATE>","<CLOSE>","<VOL>"]]
    new_table.set_index(['<TICKER>','<DATE>'],inplace=True)
    new_table
    return new_table

# get_new_table(delta_start=-30,delta_end=-20)

# Закачка последних

In [4]:
new_table = get_new_table()
new_table

<CLOSE>  <VOL>
<TICKER>     <DATE>                     
Gold         2022-12-28  1804.268      0
             2022-12-29  1814.445      0
             2022-12-30  1823.370      0
             2022-12-31  1823.580      0
             2023-01-01  1823.580      0
...                           ...    ...
@Natural gas 2023-01-03     4.014  31015
             2023-01-04     4.155  26862
             2023-01-05     3.756  29042
             2023-01-06     3.734  28979
             2023-01-07     3.734      0

[99 rows x 2 columns]

In [5]:
concat_table = new_table

# Обновление существующих

In [6]:
old_table.update(new_table)

# Сохранение

In [7]:
concat_table.reset_index(inplace=True)
concat_table

,<TICKER>,<DATE>,<CLOSE>,<VOL>
0,Gold,2022-12-28,1804.268,0
1,Gold,2022-12-29,1814.445,0
2,Gold,2022-12-30,1823.370,0
3,Gold,2022-12-31,1823.580,0
4,Gold,2023-01-01,1823.580,0
...,...,...,...,...
94,@Natural gas,2023-01-03,4.014,31015
95,@Natural gas,2023-01-04,4.155,26862
96,@Natural gas,2023-01-05,3.756,29042
97,@Natural gas,2023-01-06,3.734,28979


In [8]:
concat_table.to_csv("data.csv",index=False)

# Подкачка старых

In [9]:
concat_table = old_table.copy()
for de in range(1000,2000,20):
    new_table = get_new_table(delta_start=-de-20,delta_end=-de)
    concat_table = pd.concat([concat_table,new_table[~new_table.index.isin(concat_table.index)]])
    print(-de-20,-de)
#     break
concat_table.sort_index(inplace=True)
concat_table.reset_index().to_csv("data.csv",index=False)
concat_table

-1020 -1000
-1040 -1020
-1060 -1040
-1080 -1060
-1100 -1080
-1120 -1100
-1140 -1120
-1160 -1140
-1180 -1160
-1200 -1180
-1220 -1200
-1240 -1220
-1260 -1240
-1280 -1260
-1300 -1280
-1320 -1300
-1340 -1320
-1360 -1340
-1380 -1360
-1400 -1380
-1420 -1400
-1440 -1420
-1460 -1440
-1480 -1460
-1500 -1480
-1520 -1500
-1540 -1520
-1560 -1540
-1580 -1560
-1600 -1580
-1620 -1600
-1640 -1620
-1660 -1640
-1680 -1660
-1700 -1680
-1720 -1700
-1740 -1720
-1760 -1740
-1780 -1760
-1800 -1780
-1820 -1800
-1840 -1820
-1860 -1840
-1880 -1860
-1900 -1880
-1920 -1900
-1940 -1920
-1960 -1940
-1980 -1960
-2000 -1980


<CLOSE>    <VOL>
<TICKER>         <DATE>                      
@Brent Crude Oil 2017-07-17   48.370  22907.0
                 2017-07-18   48.730  24956.0
                 2017-07-19   49.650  23396.0
                 2017-07-20   49.320  23962.0
                 2017-07-21   47.820  26005.0
...                              ...      ...
Silver           2023-01-03   23.963      0.0
                 2023-01-04   23.776      0.0
                 2023-01-05   23.228      0.0
                 2023-01-06   23.827      0.0
                 2023-01-07   23.827      0.0

[17999 rows x 2 columns]